In [1]:
import os
import json
import pandas as pd
import traceback

In [2]:
from langchain.chat_models import ChatOpenAI



In [3]:
from dotenv import load_dotenv

load_dotenv()  # take environment variables from .env.

True

In [4]:
KEY=os.getenv("OPENAI_API_KEY") 


In [5]:
llm=ChatOpenAI(openai_api_key=KEY,model_name="gpt-3.5-turbo", temperature=0.5)

/Users/guest/mcqgen/llm_env/lib/python3.8/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [6]:
llm

ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x7fa976b69d00>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x7fa976b76b80>, temperature=0.5, openai_api_key='sk-proj-o8cwKUMdHymgyWAaTb4nT3BlbkFJlGK0tz533yCwpVUcsHrt', openai_proxy='')

In [7]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2


In [8]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}


In [9]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

# these variables below are what the user will pass :
# text is the data we want to get derive info from
# tone is difficulty level i.e intermediate, difficult, hard
# number is how many questions will be generated
# subject e.g math
# format to generate the quiz

In [10]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=TEMPLATE
    )


In [11]:
quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)

/Users/guest/mcqgen/llm_env/lib/python3.8/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [12]:
# this template is to get the correct answer for the questions from the first template i.e quiz evaluation

TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [13]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE)

In [14]:
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

# verbose return results of steps on the screen during execution phase.

In [15]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"],
                                        output_variables=["quiz", "review"], verbose=True,)

In [16]:
file_path=r"/Users/guest/mcqgen/data.txt"

In [17]:
file_path

'/Users/guest/mcqgen/data.txt'

In [18]:
with open(file_path, 'r') as file:
    TEXT = file.read()

In [19]:
print(TEXT)

Biology is the scientific study of life.[1][2][3] It is a natural science with a broad scope but has several unifying themes that tie it together as a single, coherent field.[1][2][3] For instance, all organisms are made up of cells that process hereditary information encoded in genes, which can be transmitted to future generations. Another major theme is evolution, which explains the unity and diversity of life.[1][2][3] Energy processing is also important to life as it allows organisms to move, grow, and reproduce.[1][2][3] Finally, all organisms are able to regulate their own internal environments.[1][2][3][4][5]

Biologists are able to study life at multiple levels of organization,[1] from the molecular biology of a cell to the anatomy and physiology of plants and animals, and evolution of populations.[1][6] Hence, there are multiple subdisciplines within biology, each defined by the nature of their research questions and the tools that they use.[7][8][9] Like other scientists, bio

In [20]:
# Serialize the Python dictionary into a JSON-formatted string
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [21]:
NUMBER=5 
SUBJECT="biology"
TONE="simple"


In [22]:
#https://python.langchain.com/docs/modules/model_io/llms/token_usage_tracking

#How to setup Token Usage Tracking in LangChain 
# (this allows you to track the takens and price)
with get_openai_callback() as cb:
    response=generate_evaluate_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject":SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
        )

/Users/guest/mcqgen/llm_env/lib/python3.8/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:Biology is the scientific study of life.[1][2][3] It is a natural science with a broad scope but has several unifying themes that tie it together as a single, coherent field.[1][2][3] For instance, all organisms are made up of cells that process hereditary information encoded in genes, which can be transmitted to future generations. Another major theme is evolution, which explains the unity and diversity of life.[1][2][3] Energy processing is also important to life as it allows organisms to move, grow, and reproduce.[1][2][3] Finally, all organisms are able to regulate their own internal environments.[1][2][3][4][5]

Biologists are able to study life at multiple levels of organization,[1] from the molecular biology of a cell to the anatomy and physiology of plants and animals, and evolution of populations.[1][6] Hence, there are multiple subdisciplines within biology, each define

In [23]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

Total Tokens:1933
Prompt Tokens:1192
Completion Tokens:741
Total Cost:0.00327


In [24]:
response

{'text': 'Biology is the scientific study of life.[1][2][3] It is a natural science with a broad scope but has several unifying themes that tie it together as a single, coherent field.[1][2][3] For instance, all organisms are made up of cells that process hereditary information encoded in genes, which can be transmitted to future generations. Another major theme is evolution, which explains the unity and diversity of life.[1][2][3] Energy processing is also important to life as it allows organisms to move, grow, and reproduce.[1][2][3] Finally, all organisms are able to regulate their own internal environments.[1][2][3][4][5]\n\nBiologists are able to study life at multiple levels of organization,[1] from the molecular biology of a cell to the anatomy and physiology of plants and animals, and evolution of populations.[1][6] Hence, there are multiple subdisciplines within biology, each defined by the nature of their research questions and the tools that they use.[7][8][9] Like other sci

In [25]:
quiz=response.get("quiz")

In [26]:

json.loads(quiz)


{'1': {'mcq': 'What is biology?',
  'options': {'a': 'The study of rocks',
   'b': 'The scientific study of life',
   'c': 'The study of outer space',
   'd': 'The study of economics'},
  'correct': 'b'},
 '2': {'mcq': 'What is a major theme in biology that explains the unity and diversity of life?',
  'options': {'a': 'Photosynthesis',
   'b': 'Evolution',
   'c': 'Gravity',
   'd': 'Chemical reactions'},
  'correct': 'b'},
 '3': {'mcq': 'What is the basic unit of life that all organisms are made up of?',
  'options': {'a': 'Atoms', 'b': 'Cells', 'c': 'Molecules', 'd': 'Organs'},
  'correct': 'b'},
 '4': {'mcq': 'What process allows organisms to move, grow, and reproduce?',
  'options': {'a': 'Photosynthesis',
   'b': 'Respiration',
   'c': 'Evolution',
   'd': 'Digestion'},
  'correct': 'b'},
 '5': {'mcq': 'How long ago did life on Earth emerge?',
  'options': {'a': '100 million years ago',
   'b': '1 billion years ago',
   'c': '3.7 billion years ago',
   'd': '10 billion years ago'

In [27]:
quiz=json.loads(quiz)

In [28]:
# # create a dataframe from the above dictionary.

quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [29]:
quiz_table_data

[{'MCQ': 'What is biology?',
  'Choices': 'a: The study of rocks | b: The scientific study of life | c: The study of outer space | d: The study of economics',
  'Correct': 'b'},
 {'MCQ': 'What is a major theme in biology that explains the unity and diversity of life?',
  'Choices': 'a: Photosynthesis | b: Evolution | c: Gravity | d: Chemical reactions',
  'Correct': 'b'},
 {'MCQ': 'What is the basic unit of life that all organisms are made up of?',
  'Choices': 'a: Atoms | b: Cells | c: Molecules | d: Organs',
  'Correct': 'b'},
 {'MCQ': 'What process allows organisms to move, grow, and reproduce?',
  'Choices': 'a: Photosynthesis | b: Respiration | c: Evolution | d: Digestion',
  'Correct': 'b'},
 {'MCQ': 'How long ago did life on Earth emerge?',
  'Choices': 'a: 100 million years ago | b: 1 billion years ago | c: 3.7 billion years ago | d: 10 billion years ago',
  'Correct': 'c'}]

In [33]:
pd.DataFrame(quiz_table_data)

,MCQ,Choices,Correct
0,What is biology?,a: The study of rocks | b: The scientific stud...,b
1,What is a major theme in biology that explains...,a: Photosynthesis | b: Evolution | c: Gravity ...,b
2,What is the basic unit of life that all organi...,a: Atoms | b: Cells | c: Molecules | d: Organs,b
3,"What process allows organisms to move, grow, a...",a: Photosynthesis | b: Respiration | c: Evolut...,b
4,How long ago did life on Earth emerge?,a: 100 million years ago | b: 1 billion years ...,c


In [30]:
quiz=pd.DataFrame(quiz_table_data)

In [31]:
quiz.to_csv("machinelearning.csv",index=False)

In [ ]:
from datetime import datetime
datetime.now().strftime('%m_%d_%Y_%H_%M_%S')